# infobanca

## Data understanding

### Data description report

En la exploración de los datos nos encontramos con una base de datos que contiene registros de 47871 clientes de tarjetas de un banco que maneja tres franquicias: visa, mastercard y una independiente del banco. Las variables que nos encontramos en cada uno de estos registros nos indican pertenencia a un grupo (del cual no tenemos información), estadística sobre las compras del usuario, la franquicia que usa, uso nacional o internacional, hora del día, día de la semana y tipo de comercio donde más compras realizó, totalizando 25 variables. 

El formato predominante de los datos es el formato de porcentaje, pues es así como está determinado el uso de franquicias sobre el uso total de todas, el uso nacional o internacional con las franquicias y en el total, el uso en cada rango horario y el uso en cada día de la semana; por otro lado, los datos como el mínimo, máximo y promedio de compra se encuentra en términos nominales de dinero, presumiblemente pesos colombianos y el tipo de comercio preferido como una variable categórica.

### Data exploration

Para la exploración de datos decidimos separar los registros en función del sitio preferido de los clientes, pues graficar en función de cada registro resulta inviable por la cantidad que tenemos y además carente de significado para el análisis, en cambio, los sitios preferidos si nos dan una visión significativa para las demás variables que tenemos. Por lo tanto será así como exploremos los datos y los trabajemos más adelante, como forma de exploración hicimos previamente un gráfico de barras donde se representa el valor total de compras en cada comercio usando visas en territorio nacional:

<img src='Graficoinicial.png'>

Aquí vemos entonces que con visas en territorio nacional los clientes del banco usan mayormente sus tarjetas para comprar en SUPERMERCADOS Y TIENDAS EXPRESS, ALMACEN POR DEPARTAMENTO CON SUPERMERCADO, ALMACENES DE VESTUARIO Y ACCESORIO PARA LA FAMILIA, ALAMCENES DE CALZADO, etc.

Este modo de ver la información nos resulta particularmente útil para poder sugerir al banco que tipo de promociones lanzar, pues podremos determinar que combinación de variables resultará más efectiva por la cantidad de clientes interesados en aprovecharlas gracias a esta observación y análisis de su comportamiento.

### Data quality

No encontramos errores para corregir en la base de datos en sí, sin embargo hay varibles que no serán tomadas en cuenta para el desarrollo del análisis de los datos por su carencia de significado para el mismo, tal como el grupo del cliente, el mínimo, máximo y desviación estándar de las compras, porcentaje nacional e internacional total, debido a que ya tenemos la misma información con las franquicias y de los sitios de consumo excluiremos "SIN NOMBRE".

In [16]:
import pandas as pd
import hvplot.pandas
from path import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

Importo DF

In [2]:
infobanca= pd.read_excel('infobanca.xlsx')
infobanca_original=infobanca

Creamos columna de total de ventas multiplicando el promedio por transacción por el número de transacciones

In [3]:
infobanca['Total_ventas']=infobanca['promedio_por_transaccion']*infobanca['Numero_de_transacciones']

Eliminamos columnas que no sirven pa una verga y los registros cuyo sitio favorito haya sido 'SIN NOMBRE' porque no da información que sirva pa una verga

In [4]:
coldt=['CLIENTE','grupo_de_cliente','Numero_de_transacciones','promedio_por_transaccion','transaccion_minima','transaccion_maxima','desviacion_estandar_por_transaccion','porcentaje_nacional_total','porcentaje_internacional_total']
infobanca= infobanca.drop(coldt, axis=1)
infobanca= infobanca[infobanca['Sitio_consumo_masfrecuente'] !='SIN NOMBRE']
infobanca.head(5)

,porcentaje_visa_nacional,porcentaje_visa_internacional,porcentaje_mastercard_nacional,porcentaje_mastercard_internacional,Porcentaje_otrafranquicia_nacional,porcentaje_otrafranquicia_internacional,porcentaje_manana,porcentaje_tarde,porcentaje_noche,porcDOMINGO,porcLUNES,porcMARTES,porcMIERCOLES,porcJUEVES,porcVIERNES,porcSABADO,Sitio_consumo_masfrecuente,Total_ventas
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,1.0,0.0,0.000000,0.0,0.0,CLINICAS - HOSPITALES,459930.3
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.666667,0.0,0.0,0.333333,0.0,0.0,MERCADEO DIRECTO - COMERCIANTES DE VENTAS TELE...,1748000.1
2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,1.0,0.000000,0.0,0.0,"DROGUERIAS, FARMACIAS, TIENDAS NATURISTAS",1849641.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.4,0.6,0.0,0.0,0.600000,0.2,0.0,0.100000,0.0,0.1,ALMACEN POR DEPARTAMENTO CON SUPERMERCADO,1444671.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.000000,0.0,0.0,0.000000,0.0,0.0,ALMACEN POR DEPARTAMENTO CON SUPERMERCADO,321764.0


In [12]:
infobanca

,porcentaje_visa_nacional,porcentaje_visa_internacional,porcentaje_mastercard_nacional,porcentaje_mastercard_internacional,Porcentaje_otrafranquicia_nacional,porcentaje_otrafranquicia_internacional,porcentaje_manana,porcentaje_tarde,porcentaje_noche,porcDOMINGO,porcLUNES,porcMARTES,porcMIERCOLES,porcJUEVES,porcVIERNES,porcSABADO,Sitio_consumo_masfrecuente,Total_ventas
0,0.00,459930.30,0.00,0.0,0.0,0.0,0.00,0.00,459930.30,0.00,0.0,459930.30,0.0,0.00,0.00,0.00,CLINICAS - HOSPITALES,459930.3
1,1748000.10,0.00,0.00,0.0,0.0,0.0,0.00,1748000.10,0.00,0.00,1165333.4,0.00,0.0,582666.70,0.00,0.00,MERCADEO DIRECTO - COMERCIANTES DE VENTAS TELE...,1748000.1
2,1849641.00,0.00,0.00,0.0,0.0,0.0,1849641.00,0.00,0.00,0.00,0.0,0.00,1849641.0,0.00,0.00,0.00,"DROGUERIAS, FARMACIAS, TIENDAS NATURISTAS",1849641.0
3,0.00,0.00,1444671.00,0.0,0.0,0.0,577868.40,866802.60,0.00,0.00,866802.6,288934.20,0.0,144467.10,0.00,144467.10,ALMACEN POR DEPARTAMENTO CON SUPERMERCADO,1444671.0
4,321764.00,0.00,0.00,0.0,0.0,0.0,0.00,321764.00,0.00,321764.00,0.0,0.00,0.0,0.00,0.00,0.00,ALMACEN POR DEPARTAMENTO CON SUPERMERCADO,321764.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47866,32600.00,0.00,0.00,0.0,0.0,0.0,32600.00,0.00,0.00,0.00,0.0,32600.00,0.0,0.00,0.00,0.00,VENTA DE COMIDA RAPIDA,32600.0
47867,487642.20,4876422.00,0.00,0.0,0.0,0.0,804609.63,1341016.05,3218438.52,487642.20,0.0,0.00,487642.2,2194389.90,1462926.60,731463.30,COMPRAS EN INTERNET,5364064.2
47868,172848.06,518544.18,172848.06,0.0,0.0,0.0,172848.06,345696.12,345696.12,172848.06,0.0,172848.06,0.0,172848.06,172848.06,172848.06,ALMACEN POR DEPARTAMENTO CON SUPERMERCADO,864240.3
47869,90000.00,0.00,0.00,0.0,0.0,0.0,0.00,90000.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,90000.00,ALMACENES DE VESTUARIO Y ACCESORIOS PARA TODA ...,90000.0


Multiplicamos total venta por los porcentajes del DF

In [11]:
colum = infobanca.columns
for i in colum:
  infobanca[i] = infobanca['Total_ventas']*infobanca[i]
infobanca

TypeError: can't multiply sequence by non-int of type 'float'

In [13]:
columnas=list(infobanca.columns.values)
columnas.remove('Total_ventas')

In [19]:
#suma_por_sitio=infobanca.groupby('Sitio_consumo_masfrecuente')[columnas].sum()
infobanca=suma_por_sitio

# Preparar los datos

Escalar toa esa verga

In [20]:
infobanca_escalado=StandardScaler().fit_transform(infobanca)

# Crea un DataFrame con los datos escalados
df_infobanca_scaled = pd.DataFrame(
    infobanca_escalado,
    columns=infobanca.columns
)

# Copy the crypto names from the original data
df_infobanca_scaled["Sitio_consumo_masfrecuente"] = infobanca.index

# Establecer la columna coinid como índice
df_infobanca_scaled= df_infobanca_scaled.set_index("Sitio_consumo_masfrecuente")

# Mostrar datos de muestra
df_infobanca_scaled.head()

,porcentaje_visa_nacional,porcentaje_visa_internacional,porcentaje_mastercard_nacional,porcentaje_mastercard_internacional,Porcentaje_otrafranquicia_nacional,porcentaje_otrafranquicia_internacional,porcentaje_manana,porcentaje_tarde,porcentaje_noche,porcDOMINGO,porcLUNES,porcMARTES,porcMIERCOLES,porcJUEVES,porcVIERNES,porcSABADO
Sitio_consumo_masfrecuente,,,,,,,,,,,,,,,,
"ADMINISTRACION DE AREAS COMUNES, SERVICIOS DE LIMPIEZA, MANTENIMIENTO Y CELADURIA",-0.405491,-0.288621,-0.356499,-0.295739,-0.334362,-0.159010,-0.413601,-0.397871,-0.362380,-0.318788,-0.406219,-0.426617,-0.425587,-0.428481,-0.424399,-0.364971
AEROLINEAS,4.001293,3.815421,1.414043,3.221069,0.978400,1.108298,3.111970,2.487580,3.576961,1.826473,2.898485,3.336294,3.377052,3.281474,3.558367,1.987543
AGENCIAS DE BOLETERIA ( Producciones de Teatro ) excepto cine,-0.312152,-0.255159,-0.330613,-0.277400,-0.327660,-0.153257,-0.364779,-0.351821,-0.296694,-0.262730,-0.380057,-0.372397,-0.361951,-0.375383,-0.388687,-0.302732
AGENCIAS DE PROTECCIÓN Y SERVICIOS DE SEGURIDAD,-0.406317,-0.288621,-0.356499,-0.295739,-0.334362,-0.159010,-0.414570,-0.397871,-0.362380,-0.318788,-0.406219,-0.429417,-0.425587,-0.428481,-0.421504,-0.366764
AGENCIAS DE VIAJE y OPERADORES DE TURISMO,2.417142,0.354322,0.292076,0.158033,0.004607,0.267747,1.194076,1.143043,0.353504,0.279540,1.247134,1.327527,1.143006,1.204936,1.272676,0.748885


## Encontrar el valor de K usando los datos originalesss

cojo números del 1 al 15

In [21]:
k = list(range(1,15))
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]

In [22]:
# Crear una lista vacía para almacenar los valores de inertia
inertia = []

In [23]:
# Crear un bucle for para calcular la inercia con cada valor posible de k
# Dentro del bucle:
# 1. Cree un modelo KMeans usando el contador de bucles para los n_clusters
# 2. Ajuste el modelo a los datos usando `df_market_data_scaled`
# 3. Agrega model.inertia_ a la lista de inercias

for i in k:
    model = KMeans(n_clusters = i, random_state = 0)
    model.fit(df_infobanca_scaled)
    inertia.append(model.inertia_)

inertia

c:\Users\program\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\program\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\program\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\program\AppData\Local\Programs\Python\Python311\Lib\site-packages\skle

[1728.0,
 695.8360241609633,
 431.4107157755393,
 297.43829595634327,
 198.18593544481575,
 127.10644876584452,
 90.23440399770587,
 58.09151619601399,
 42.11421287634237,
 33.81113967766838,
 27.52011927243752,
 21.81994402685451,
 17.528663597986558,
 13.799700854574656]

In [24]:
# Crear un diccionario con los datos para trazar la curva del Codo
elbow_data_original = {
    'k': k,
    'inertia': inertia
}

# Crea un DataFrame con los datos para trazar la curva del Codo
df_elbow_data_original = pd.DataFrame(elbow_data_original)
df_elbow_data_original

,k,inertia
0,1,1728.000000
1,2,695.836024
2,3,431.410716
3,4,297.438296
4,5,198.185935
5,6,127.106449
6,7,90.234404
7,8,58.091516
8,9,42.114213
9,10,33.811140


In [25]:
# Hacer un gráfico de líneas con todos los valores de inercia calculados con
# los diferentes valores de k para identificar visualmente el valor óptimo para k.
elbow_plot_original = df_elbow_data_original.hvplot.line(
    title = 'Elbow Plot: Original Data',
    x = 'k',
    xlabel = 'k',
    xticks = k,
    y = 'inertia',
    ylabel = 'Inertia'
)

elbow_plot_original

:Curve   [k]   (inertia)

cojo el k=6

## Clusters

In [28]:
# Inicializar el modelo K-Means usando el mejor valor para k
model = KMeans(n_clusters=6)

In [29]:
# Ajuste el modelo K-Means usando los datos escalados
model.fit(df_infobanca_scaled)

c:\Users\program\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=6)

In [30]:
# Prediga los grupos para agrupar las criptomonedas usando los datos escalados
clusters_original = model.predict(df_infobanca_scaled)

# Ver la matriz resultante de valores de clúster.
clusters_original

array([1, 2, 1, 1, 3, 1, 1, 2, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 2, 1, 3, 1, 4, 1, 3, 1, 1, 1, 1, 3, 1, 1, 5, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1,
       1, 3, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1])

In [31]:
# Crear una copia del DataFrame
df_clusters_original = df_infobanca_scaled.copy()

In [32]:
# Agregue una nueva columna al DataFrame con los grupos predichos
df_clusters_original['Cluster'] = clusters_original

# Mostrar datos de muestra
df_clusters_original

,porcentaje_visa_nacional,porcentaje_visa_internacional,porcentaje_mastercard_nacional,porcentaje_mastercard_internacional,Porcentaje_otrafranquicia_nacional,porcentaje_otrafranquicia_internacional,porcentaje_manana,porcentaje_tarde,porcentaje_noche,porcDOMINGO,porcLUNES,porcMARTES,porcMIERCOLES,porcJUEVES,porcVIERNES,porcSABADO,Cluster
Sitio_consumo_masfrecuente,,,,,,,,,,,,,,,,,
"ADMINISTRACION DE AREAS COMUNES, SERVICIOS DE LIMPIEZA, MANTENIMIENTO Y CELADURIA",-0.405491,-0.288621,-0.356499,-0.295739,-0.334362,-0.159010,-0.413601,-0.397871,-0.362380,-0.318788,-0.406219,-0.426617,-0.425587,-0.428481,-0.424399,-0.364971,1
AEROLINEAS,4.001293,3.815421,1.414043,3.221069,0.978400,1.108298,3.111970,2.487580,3.576961,1.826473,2.898485,3.336294,3.377052,3.281474,3.558367,1.987543,2
AGENCIAS DE BOLETERIA ( Producciones de Teatro ) excepto cine,-0.312152,-0.255159,-0.330613,-0.277400,-0.327660,-0.153257,-0.364779,-0.351821,-0.296694,-0.262730,-0.380057,-0.372397,-0.361951,-0.375383,-0.388687,-0.302732,1
AGENCIAS DE PROTECCIÓN Y SERVICIOS DE SEGURIDAD,-0.406317,-0.288621,-0.356499,-0.295739,-0.334362,-0.159010,-0.414570,-0.397871,-0.362380,-0.318788,-0.406219,-0.429417,-0.425587,-0.428481,-0.421504,-0.366764,1
AGENCIAS DE VIAJE y OPERADORES DE TURISMO,2.417142,0.354322,0.292076,0.158033,0.004607,0.267747,1.194076,1.143043,0.353504,0.279540,1.247134,1.327527,1.143006,1.204936,1.272676,0.748885,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"VENTA DE ARTICULOS MEDICOS, ODONTOLOGICOS, ORTOPEDICOS",-0.334900,-0.288621,-0.311536,-0.270095,-0.334362,-0.159010,-0.348910,-0.347951,-0.338349,-0.304006,-0.376855,-0.372506,-0.352579,-0.393466,-0.374642,-0.280720,1
VENTA DE COMIDA RAPIDA,-0.309190,-0.238212,-0.242338,-0.244984,-0.294081,-0.147966,-0.348767,-0.281156,-0.234774,-0.166251,-0.293692,-0.338087,-0.339348,-0.328613,-0.334026,-0.260543,1
VENTA DE BOTES Y ACCESORIOS PARA ACTIVIDADES ACUATICAS,-0.405782,-0.288621,-0.351139,-0.295739,-0.334362,-0.159010,-0.410359,-0.394422,-0.362380,-0.315946,-0.399186,-0.429417,-0.423474,-0.427248,-0.424399,-0.356686,1


In [36]:
cluster_plot_original = df_clusters_original.hvplot.scatter(
    title='Cluster Plot: Original',
    x = 'porcentaje_manana',
    xlabel = 'porcentaje_manana',
    y = 'porcSABADO',
    ylabel = 'porcSABADO',
    by = 'Cluster',
    hover_cols=["Sitio_consumo_masfrecuente"]
)

cluster_plot_original

:NdOverlay   [Cluster]
   :Scatter   [porcentaje_manana]   (porcSABADO,Sitio_consumo_masfrecuente)